# Weight Decay

**Weight Decay（重み減衰）** は，モデルのパラメータの大きさを小さく保つよう勾配降下の過程で重みを直接減少させる手法である．具体的には、Weight Decayは$d$ 次元のパラメータ $\boldsymbol{\theta}$ を持つ損失関数 $L(\boldsymbol{\theta})$ に対して，各重み更新の際に以下のように減少項を適用する.

$$
\theta_i \leftarrow \theta_i - \eta \frac{\partial L}{\partial \theta_i} - \eta \lambda \theta_i
$$

ここで，$\eta$ は学習率，$\lambda$ はweight decayの強度を表すハイパーパラメータである．

このように直接的にパラメータの値を減衰させることで，パラメータの大きさを小さく保ち，過学習を抑えることができる．非常にシンプルなアイデアであるが，深層学習においても重要な手法であり，学習曲線を確認して過学習気味であれば適用すると良い．



微分を見ても明らかであるように，確率的勾配降下法でパラメータを更新する際に，Weight Decay項を考えれば良いことがわかる．PyTorchでは，最適化を行う `torch.optim` の中にWeight Decayに関する実装が含まれており，以下のようにすればWeight Decayを適用した確率的勾配降下法によるパラメータ更新が行える．

```
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=weight_decay)
```

## $L_2$ 正則化

**$L_2$ 正則化** は，weight decayと同様にモデルのパラメータの大きさを小さく保つようにすることで過学習を抑制する正則化手法の一種である．具体的に，パラメータの大きさに関するペナルティ項を損失に加える．$d$ 次元のパラメータ $\boldsymbol{\theta}$ を持つ損失関数 $L(\boldsymbol{\theta})$ に対して，次のようなパラメータの二乗和をペナルティ項として追加する．

$$
L'(\boldsymbol{\theta}) = L(\boldsymbol{\theta}) + \frac{\lambda}{2} \sum^d_{i=1} \theta_i^2
$$

ここで，$\lambda$ はペナルティの強度を調整するハイパーパラメータである．

全体の損失 $L'(\boldsymbol{\theta})$ を小さくするには，オリジナルの損失 $L(\boldsymbol{\theta})$ だけでなくパラメータの値を小さくしなければ第二項目が大きくなってしまう．これによって，最適化の際に，パラメータの値が小さく抑えられる．

全体の損失 $L'(\boldsymbol{\theta})$ をパラメータで微分してみると，

$$
\frac{\partial L'}{\partial \theta_i} = \frac{\partial L}{\partial \theta_i}+\lambda \theta_i
$$

となる．右辺の第一項は勾配である．これを更新式に当てはめると，

$$
\begin{align}
\theta_i &\leftarrow \theta_i - \eta \frac{\partial L'}{\partial \theta_i} \\
&= \theta_i - \eta \left ( \frac{\partial L}{\partial \theta_i}+\lambda \theta_i \right) \\
&= \theta_i - \eta \frac{\partial L}{\partial \theta_i} - \eta \lambda \theta_i
\end{align}
$$

となり，勾配にパラメータを $\lambda$ 倍した値を加算すれば良いこととなる．そして，Weight Decayの更新式と一致しており，Weight Decayと$L_2$正則化が等価であることがわかる．

## 注意

この等価性からWeight Decayと$L_2$正則化を同じものとして説明している記事や実装があるが，Weight DecayとL_2正則化が等価であるのは上記のような更新式を用いた場合のみ，つまり，SGDの場合のみに限られる．実際，PyTorchのAdamと呼ばれるオプティマイザはこの等価性の勘違いから上記の手順でAdam w/ Weight Decayを実装していた．Adamではこの等価性を満たさないこと，AdamがWeight Decayの恩恵を得られていないことが[論文](https://arxiv.org/abs/1711.05101v3)で指摘され，AdamWというViTの学習で標準的なオプティマイザが提案されている．